In [ ]:
## Here I am removing all the protected features to see the difference from normal

import pandas as pd
import random,time
import numpy as np
import math,copy
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import sklearn.metrics as metrics


from Measure import measure_final_score,calculate_recall,calculate_far,calculate_precision,calculate_accuracy

In [ ]:
## Load dataset
from sklearn import preprocessing
dataset_orig = pd.read_csv('dataset/processed.cleveland.data.csv')

## Drop NULL values
dataset_orig = dataset_orig.dropna()


## calculate mean of age column
mean = dataset_orig.loc[:,"age"].mean()
dataset_orig['age'] = np.where(dataset_orig['age'] >= mean, 1, 0)

## Make goal column binary
dataset_orig['Probability'] = np.where(dataset_orig['Probability'] > 0, 1, 0)


from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
dataset_orig = pd.DataFrame(scaler.fit_transform(dataset_orig),columns = dataset_orig.columns)


# dataset_orig

# Results with protected attributes

In [ ]:
np.random.seed(0)
## Divide into train,validation,test
dataset_orig_train, dataset_orig_test = train_test_split(dataset_orig, test_size=0.2, random_state=0,shuffle = True)

X_train, y_train = dataset_orig_train.loc[:, dataset_orig_train.columns != 'Probability'], dataset_orig_train['Probability']
X_test , y_test = dataset_orig_test.loc[:, dataset_orig_test.columns != 'Probability'], dataset_orig_test['Probability']

# --- LSR
clf = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100)
# --- CART
# clf = tree.DecisionTreeClassifier()

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
cnf_matrix_test = confusion_matrix(y_test,y_pred)

print(cnf_matrix_test)
TN, FP, FN, TP = confusion_matrix(y_test,y_pred).ravel()


print("recall :", measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'age', 'recall'))
print("far :",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'age', 'far'))
print("precision :", measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'age', 'precision'))
print("accuracy :",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'age', 'accuracy'))
print("aod age:",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'age', 'aod'))
print("eod age:",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'age', 'eod'))


print("Precision", metrics.precision_score(y_test,y_pred))
print("Recall", metrics.recall_score(y_test,y_pred))
print(X_train.columns)
print(clf.coef_)


import matplotlib.pyplot as plt
y = np.arange(len(dataset_orig.columns)-1)
plt.barh(y,clf.coef_[0])
plt.yticks(y,dataset_orig_train.columns)
plt.show()

# Results without protected attributes

In [ ]:
## Drop age
dataset_orig = dataset_orig.drop(['age'],axis=1)


np.random.seed(0)
## Divide into train,validation,test
dataset_orig_train, dataset_orig_test = train_test_split(dataset_orig, test_size=0.2, random_state=0,shuffle = True)

X_train, y_train = dataset_orig_train.loc[:, dataset_orig_train.columns != 'Probability'], dataset_orig_train['Probability']
X_test , y_test = dataset_orig_test.loc[:, dataset_orig_test.columns != 'Probability'], dataset_orig_test['Probability']

# --- LSR
clf = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100)
# --- CART
# clf = tree.DecisionTreeClassifier()

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
cnf_matrix_test = confusion_matrix(y_test,y_pred)

print(cnf_matrix_test)
TN, FP, FN, TP = confusion_matrix(y_test,y_pred).ravel()

print("recall :", calculate_recall(TP,FP,FN,TN))
print("far :",calculate_far(TP,FP,FN,TN))
print("precision :", calculate_precision(TP,FP,FN,TN))
print("accuracy :",calculate_accuracy(TP,FP,FN,TN))


print(X_train.columns)
print(clf.coef_)